In [1]:
# Load the Excel file

import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment

file_path = '/Users/jacksonsiemens/Desktop/R/Basebol/data/orioles_lineups.xlsx'
player_data = pd.read_excel(file_path)

# Display the first few rows of the dataframe to understand its structure
player_data.head()

,id,Name,bats,bat,bat_sd,plat,bat_vsRHP,bat_vsLHP,C,1B,2B,3B,SS,LF,CF,RF
0,BAL,Cedric Mullins,L,3.7,19,18,12.7,-5.3,NaN,NaN,NaN,NaN,NaN,15.0,8.0,13.0
1,BAL,Austin Hays,R,1.3,9,-8,-2.7,5.3,NaN,NaN,NaN,NaN,NaN,6.0,-4.0,8.0
2,BAL,Colton Cowser,L,0.3,11,18,9.3,-8.7,NaN,NaN,NaN,NaN,NaN,7.0,1.0,5.0
3,BAL,Ryan McKenna,R,-1.0,21,-3,-2.5,0.5,NaN,NaN,NaN,NaN,NaN,7.0,3.0,4.0
4,BAL,Aaron Hicks,S,-2.4,19,4,-0.4,-4.4,NaN,NaN,NaN,NaN,NaN,4.0,-3.0,3.0


In [2]:
# For defensive projections, we are saying per 162 games. Equating to 1458 Innings.
# When calculating a players defensive value at various positions, I'm going to scale their past performance to reflect a 1458 Innings total.
# This feels inherently flawed, but after testing around with some orioles projections and Statcast Defensive RAR, it seems pretty accurate

def defensive_pace_pos(defensive_dict):
    # Dictionary to hold the pace values for each position
    pace_dict = {}

    # Iterate over the keys in the dictionary
    for key in defensive_dict.keys():
        # Identify the position and corresponding metric (innings or RAR)
        if 'innings' in key:
            position = key.split('_')[0]
            innings = sum(defensive_dict[key])
            # Check if RAR for the same position exists
            if f'{position}_RAR' in defensive_dict:
                rar = sum(defensive_dict[f'{position}_RAR'])
                # Calculate the pace for this position
                if innings > 0:
                    pace = rar / innings * 1458
                    pace_dict[position] = pace

    return pace_dict

# Test the function with the provided data
frelick = {
    'CF_innings': [196.0], 
    'CF_RAR': [2], 
    'RF_innings': [246.1], 
    'RF_RAR': [4], 
}

frelick_defence = defensive_pace_pos(frelick)
frelick_defence 

{'CF': 14.877551020408163, 'RF': 23.697683868346203}

This is absurd. To get his offensive contribution, 

In [3]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder

file_path = 'data/fg_leaders.csv'
df = pd.read_csv(file_path)


# Scale 'Off' to 600 PAs
df['Scaled_Off'] = df['Off'] * (600 / df['PA'])

# One-hot encode 'Team'
encoder = OneHotEncoder(sparse=False)
team_encoded = encoder.fit_transform(df[['Team']])
team_df = pd.DataFrame(team_encoded, columns=encoder.get_feature_names_out(['Team']))

# Merge features
X = pd.concat([df[['AVG', 'OBP', 'SLG']], team_df], axis=1)
y = df['Scaled_Off']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=4, min_samples_leaf=4, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)
train_r2 = r2_score(y_train, train_pred)
test_r2 = r2_score(y_test, test_pred)

print(f'Train R²: {train_r2}, Test R²: {test_r2}')


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Train R²: 0.9513003328783476, Test R²: 0.9436792035055751


Model is super good, which makes sense.

In [4]:
def predict_off_value(team, avg, obp, slg, model, encoder):
    # One-hot encode the team
    team_encoded = encoder.transform([[team]])
    team_df = pd.DataFrame(team_encoded, columns=encoder.get_feature_names_out())

    # Create a DataFrame for the player's stats
    player_stats = pd.DataFrame([[avg, obp, slg]], columns=['AVG', 'OBP', 'SLG'])
    
    # Combine player stats with team data
    features = pd.concat([player_stats, team_df], axis=1)

    # Predict the 'Off' value
    predicted_off = model.predict(features)

    return predicted_off[0]

## Sal Frelick
## These projections are pulled from ZIPS
# RHP
print("RHP: ", predict_off_value("MIL",0.275,0.335, 0.392, model, encoder))

#LHP
print("LHP: ", predict_off_value("MIL",0.267,0.328, 0.378, model, encoder))



RHP:  0.5964567496379495
LHP:  -5.01049749422324


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [5]:
ops_plus_80th = 115
ops_plus_20th = 81
ops_plus_mean = 99

# Assuming a symmetrical distribution, the standard deviation can be approximated
# by the average distance of the 80th and 20th percentiles from the mean
std_dev_approx = (abs(ops_plus_80th - ops_plus_mean) + abs(ops_plus_mean - ops_plus_20th)) / 2
std_dev_approx

17.0

Mean Projection : -1.1945825908574963
Against LHP:  -5.01049749422324
Against RHP: 0.5964567496379495

STD : 17.0

In [6]:
#{'CF': 14.877551020408163, 'RF': 23.697683868346203}

frelick = {
    'id': 664056,
    'Name': 'Sal Frelick',
    'bats': 'R',
    'bat': -1.1945825908574963,
    'bat_sd': 17.0,
    'plat': 0,
    'bat_vsRHP': 0.5964567496379495,
    'bat_vsLHP': -5.01049749422324,
    'C': np.nan,     # Assuming NaN should be represented by np.nan
    '1B': np.nan,
    '2B': np.nan,
    '3B': np.nan,
    'SS': np.nan,
    'LF': 14, # No way it's less than this with respect to the process
    'CF': 14.87755102040816,
    'RF': 23.697683868346203
}
player_data_w_frelick = player_data.append(frelick, ignore_index=True)
player_data_w_frelick.fillna(-1000, inplace=True)

# Function to optimize lineup including DH
def optimize_lineup_with_dh(player_data, against_pitcher_type):
    bat_column = f'bat_vs{against_pitcher_type}'
    sd_column = 'bat_sd'  # Column name for standard deviation
    positions = ['C', '1B', '2B', '3B', 'SS', 'LF', 'CF', 'RF']

    total_performance_matrix = np.full((len(player_data), len(positions) + 1), -1000.0)  # Include DH

    # Fill the matrix for position players
    for i, position in enumerate(positions):
        can_play = player_data[position] > -1000
        total_performance_matrix[can_play, i] = player_data[bat_column][can_play] + player_data[position][can_play]

    # Fill the matrix for DH (only offensive contribution)
    total_performance_matrix[:, -1] = player_data[bat_column]

    # Optimization using linear sum assignment
    row_ind, col_ind = linear_sum_assignment(-total_performance_matrix)  # Negative for maximization

    # Calculate total optimized runs above average
    optimized_runs = total_performance_matrix[row_ind, col_ind].sum()

    # Create the optimized lineup
    optimized_lineup = player_data.iloc[row_ind]
    positions.append('DH')  # Add DH to position list
    optimized_lineup['Position'] = [positions[i] for i in col_ind]

    # Calculate SD of the total RAA of the lineup
    # SD calculation assumes independence of player performances
    lineup_sd = np.sqrt((player_data.iloc[row_ind][sd_column] ** 2).sum())

    return optimized_lineup, optimized_runs, lineup_sd



/var/folders/8n/g8bq8_011kv24tnfb649vgmw0000gn/T/ipykernel_1522/3199097631.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_data_w_frelick = player_data.append(frelick, ignore_index=True)


In [7]:
optimized_lineup_rhp, total_runs_rhp, total_sd_rhp = optimize_lineup_with_dh(player_data_w_frelick, 'RHP')
optimized_lineup_lhp, total_runs_lhp, total_sd_lhp = optimize_lineup_with_dh(player_data_w_frelick, 'LHP')

/var/folders/8n/g8bq8_011kv24tnfb649vgmw0000gn/T/ipykernel_1522/3199097631.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  optimized_lineup['Position'] = [positions[i] for i in col_ind]
/var/folders/8n/g8bq8_011kv24tnfb649vgmw0000gn/T/ipykernel_1522/3199097631.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  optimized_lineup['Position'] = [positions[i] for i in col_ind]


In [20]:
# Slater
print("LHP: ", predict_off_value("SFG",0.254,0.352, 0.421, model, encoder))

# This projection seems too low on him, He hasn't posted anything lower than a .361 OBP against lefties since 2018. And his last three seasons have been .373/.379/.361.
# He hasn't hit lower than he did last year, which was a .288/.361/.439. He's been a platoon destroyer for the past three seasons. The BABIP's have been high though, so I'll role with

LHP:  10.432889402664598


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [21]:
slater = {
    'CF_innings': [181, 538, 386], 
    'CF_RAR': [1, 1, 1], 
    'RF_innings': [33, 40, 93], 
    'RF_RAR': [1, -1, 0], 
    'LF_innings': [165, 60, 123], 
    'LF_RAR': [-1, 1, 1]
}

slater_defence = defensive_pace_pos(slater )
slater_defence 

{'CF': 3.9583710407239816, 'RF': 0.0, 'LF': 4.189655172413793}

In [23]:
noot = {
    'CF_innings': [570, 80], 
    'CF_RAR': [1, 1], 
    'RF_innings': [233, 585, 181], 
    'RF_RAR': [0, -1, 1], 
    'LF_innings': [161, 46, 40], 
    'LF_RAR': [2, 0, 1]
}
defensive_pace_pos(noot)

# Generally speaking I'm extremely uncertain with the way that FG handles defensive projection. Lars has never posted a season with a negative defensive value, yet they project him well below average which makes little sense 
# I'm not giving noot a 17 in Left though, I think he's marginally above average. Regardless, he will be extreme value

{'CF': 4.486153846153846, 'RF': 0.0, 'LF': 17.708502024291498}

In [24]:
print("RHP: ", predict_off_value("STL", 0.245,0.347, 0.455, model, encoder))
print("LHP: ", predict_off_value("STL", 0.233,0.336, 0.405, model, encoder))

RHP:  14.613712196935648
LHP:  3.885227328661415


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
